За основу для экспериментов по DPO-алайменту была взята инструктированная модель [SmolLM2-135M-Instruct](https://huggingface.co/HuggingFaceTB/SmolLM2-135M-Instruct). В качестве набора данных использовался был взят [ultrafeedback_binarized](https://huggingface.co/datasets/trl-lib/ultrafeedback_binarized).


Параметры обучения

шаг обучения 5e-5
beta 0.1
bf16 = True
logging_steps = 1

max_promt_length = 1024
max_length = 1536

per_device_eval_batch_size  = 4
per_device_train_batch_size = 4
gradient_accumulation_steps = 4


max_steps                   = 200

### __Параметры для Reverse KL__

Шаг обучения $5e-5$, коэффициент KL дивергенции $\beta \in [0.05, 0.1, 1.0, 5.0]$

### __Параметры для Forward KL__

### __Параметры для $\alpha$-divergence__

$L(\theta, D) = E_{(x, y_w, y_l)\~D}\bigg[-\text{log}\sigma\bigg(\beta$

### __Как рассчитывалась KL-дивергенция__

Фиксируем 100 тестовых промтов. Двумя взятыми моделями генерируем ответы для каждого промта. На основе выходных логитов модели рассчитываем KL-дивергенцию для каждой пары ответов. В конце усредняем.